<a href="https://colab.research.google.com/github/fuluki/awesome-notebooks-SEO/blob/master/kw_headers_copia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.5 MB/s eta 0:00:00


In [ ]:

import requests
import openai
from difflib import SequenceMatcher

# Configuración
openai.api_key = "sk-0s9proocCOtUzTdXur5BTr4lbkFJ9MQfZZBvAIasGBPxJWFm"

# Funciones auxiliares
def clean_response(response):
    return response.strip()

def get_semrush_keywords(api_key, keyword, limit=15):
    url = "https://api.semrush.com"
    params = {
        "type": "phrase_fullsearch",
        "key": api_key,
        "phrase": keyword,
        "database": "co",
        "export_columns": "Ph,Nq",  # Solo queremos palabras clave y volumen de búsqueda
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        lines = response.text.split("\r\n")[1:-1]
        keywords_data = [line.split(';')[:2] for line in lines]  # Tomamos las columnas Ph y Nq
        return keywords_data[:limit]
    else:
        print(f"Error {response.status_code}: {response.text}")
        return []


# Paso 1: Solicitar palabra clave
keyword_input = input("Introduce tu palabra clave: ")

# Paso 3: Obtener palabras clave relacionadas
related_keywords = get_semrush_keywords("de559139fcf7532d34f34ff", keyword_input)

# Convertir el listado de palabras clave en una cadena formateada
keyword_list = "\n".join([f"{kw} - {volume}" for kw, volume in related_keywords])
#print(keyword_list)  # Ahora esto debería imprimir el listado de palabras clave

# Crear el prompt para GPT-3
messages = [
    {"role": "system", "content": "Eres un experto en SEO y filtrado de palabras clave."},
    {"role": "user", "content": f"Filtra y elimina las keywords con misma intención de búsqueda, que sean iguales pero con o sin acentos, que incluyan animales (perros o gatos) o que incluya una ciudad (elimina las que contengan precio, precios, alicante, madrid, valencia (u otras ciudades), seguridad social, hospitales, PDF o gratis) y preséntamelo en formato de lista:\n\n{keyword_list}"}
]

# Llamar a GPT-3 para obtener el listado filtrado
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages
)
#print(response.choices[0].message.content)  # Imprime la respuesta cruda de GPT-3

# Procesar la respuesta de GPT-3 y convertirla en una lista de tuplas
filtered_response = clean_response(response.choices[0].message.content)
final_keywords = [tuple(line.split(" - ")) for line in filtered_response.split("\n") if " - " in line]

# Muestra las palabras clave finales

for kw, volume in final_keywords:
    print(f"{kw} - {volume}")

def generate_blog_structure(keywords_data):
    # Convertir el listado de palabras clave en una cadena formateada
    keyword_list = "\n".join([f"{kw} - {volume}" for kw, volume in keywords_data])

    # Crear el prompt para GPT-3
    messages = [
        {
            "role": "system",
            "content": "Eres un experto en estructuración de contenidos para blog."
        },
        {
            "role": "user",
            "content": f"""
                Quiero que dándote un listado de palabras clave junto a su volumen de búsqueda, me generes una estructura para un post de blog. Es muy importante que no incluyas las que ya se repitan, sean innecesarias o no consideres necesarias que se incluyan en un encabezado. Tampoco es necesario que las incluyas tal cual aparecen escritas, es importante que sea natural.

                Te pongo un ejemplo de cómo debería ser:
                Este es el listado de keywords junto a su volumen de búsqueda (lo que sigue al guión "-"):
                mejores gestores de contraseñas - 210
                gestores contraseñas - 210
                gestor de contraseñas android - 90
                gestores de contraseñas gratuitos - 70
                app para guardar contraseñas - 90
                app para guardar contraseñas - 30
                gestor contraseñas iphone - 40
                gestor de contraseñas de google - 4400

                Con estos datos he creado lo siguiente:
                H2: Mejores gestores de contraseñas
                H2: Mejores APPs para guardar contraseñas
                H3: Gestores de contraseñas para Android
                H3: Gestores de contraseñas para iPhone
                H2: Gestores de contraseñas para Windows
                H2: Gestores de contraseñas gratuitos
                H2: ¿Cómo funciona el gestor de contraseñas de Google?

                Ahora te daré un nuevo listado, generame la estructura de encabezados similar y separada por saltos de línea. Dame solo 4-10 encabezados. Nunca incluyas H1. Si ves keywords muy similares agrupalas en 1 encabezado. Dame también, una idea de encabezado H1, una url (ejemplo: /gestores-contrasenas/) un meta title acorde a mi artículo que incluya las palabras clave de mayor volumen de búsqueda y una meta descripción (Recuerda respetar la longitud máxima). Mi lista de palabras es:
                {keyword_list}
            """
        }
    ]

    # Llamar a GPT-3 para obtener la estructura del blog
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    return clean_response(response.choices[0].message.content)

# Usa la función
if final_keywords:  # Verificar que hay palabras clave finales
    blog_structure = generate_blog_structure(final_keywords)
    print("\nEstructura para el post de blog:")
    print(blog_structure)
else:
    print("No se encontraron palabras clave relevantes para generar una estructura de blog.")


Introduce tu palabra clave: nombre de dominio
sistema de nombres de dominio - 2900
nombre de dominio - 1600
que es un nombre de dominio - 880
nombre de dominio ejemplos - 320
nombre de dominio definicion - 140
cambiar nombre de dominio server 2016 - 90
como saber el nombre de dominio en linux - 90
cual es mi nombre de dominio - 90
nombre de dominio de red - 90
poner nombre de dominio a mi ip - 90
sistema de nombres de dominio dns definicion - 90
synology el nombre de dominio no se puede resolver - 90
validar nombre de dominio - 90
cambiar el nombre de mi dominio en godaddy - 70
cambiar nombre de dominio hostgator - 70

Estructura para el post de blog:
Basándome en las palabras clave y su volumen de búsqueda que me proporcionaste, aquí está la estructura de encabezados para tu post de blog:

H1: Guía completa sobre nombres de dominio
URL: /guia-nombres-dominio/
Meta Title: Guía completa sobre nombres de dominio: sistema, definición y ejemplos
Meta Descripción: Descubre todo sobre los no